In [2]:
from MDA_Data_preparation import df, df_s, df_s1, df_s2, b_ind, b_typ, df_sm, crosst, Factors, mca_ind
import mca
import pandas as pd
import numpy as np
from matplotlib  import dates
from sklearn import linear_model



ModuleNotFoundError: No module named 'MDA_Data_preparation'

# Viewing data structure

In [3]:
df.head(n=3)

,company_name,company_description,relationship_to_bond,sector,industry,ISIN,bond_destination,coupon_rate,total_bond_amount,nominal_amount,quarter_created,year_created,maturity_date,bond_currency,country,status,stocks_info,bond_types,Floating rate,Foreign bonds,Green bonds,Securitization,Senior Secured,Senior Unsecured,Zero-coupon bonds
0,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1088274169,International bonds,1.75%,"362,793,000 EUR","1,000 EUR",Q2,2017,7/16/2019,EUR,Italy,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no
1,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1088274672,International bonds,3%,"600,000,000 EUR","1,000 EUR",Q2,2017,7/16/2024,EUR,Italy,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,no,no,no,yes,no
2,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1144492532,International bonds,1.13%,"267,100,000 EUR","1,000 EUR",Q2,2017,1/2/2020,EUR,Italy,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no


In [1]:
# We select 9 specifyc variables for  analysis
df_s.head(n=3)

NameError: name 'df_s' is not defined

We see there are 2,083 observations with bonds

In [5]:
df_s.describe(exclude=np.number)

,industry,country,bond_types,ISIN,total_bond_amount,coupon_rate,nominal_amount,maturity_date
count,2083,2083,1206,2083,2083,2083,2083,2083
unique,20,26,15,2055,209,168,9,1611
top,Power,France,Senior Unsecured,FR0014000C08,"500,000,000 EUR",1%,"1,000 EUR",4/7/2025
freq,346,562,922,10,587,118,1119,6


In [6]:
1206/2083

0.5789726356216994

We see that only 58% of the bonds have assigned a "bond type". Since this characteristic is of importance to detect if the bank has helped or not the green economy, we will only use in the analysis bonds with a non-missing bond type, so we reduce our data set to these 1206 observations.

- Number of bonds by industry

In [7]:
b_ind.T

,Non-ferrous metals,Ferrous metals,"Pulp, paper and wood industries",Mining industry,Trade and retail,Media and Entertainment,Light industry,Other sectors,Public utilities,Information and High Technologies,Healthсare,Food industry,Transportation,Chemical and petrochemical industry,Communication,Oil and gas,Financial institutions,Construction and development,Engineering industry,Power
# Bonds,3,4,6,8,11,13,20,24,34,40,48,63,79,83,102,107,115,119,145,182


- Number of bonds by type of bond

In [8]:
b_typ.T

,"Foreign bonds, Floating rate",Securitization,"Green bonds, Senior Secured","Foreign bonds, Senior Unsecured","Zero-coupon bonds, Senior Secured","Green bonds, Zero-coupon bonds, Senior Unsecured",Foreign bonds,"Senior Unsecured, Floating rate",Senior Secured,Green bonds,Zero-coupon bonds,Floating rate,"Zero-coupon bonds, Senior Unsecured","Green bonds, Senior Unsecured",Senior Unsecured
# Bonds,1,1,2,2,2,4,6,11,21,32,39,39,45,79,922


We group industries with low observations and add them into the "other sectors" group. The purpose of this is to not draw assumptions from them in the correspondence analysis that are only based in a couple of observations. The same procedure is done for bond_types. Then, We visualize the frequency of each of the bond type by country, and the number of bonds per country. We will group countries which are similar and which have a very small number of bonds.

In [9]:
crosst

bond_types,Green bonds,Others,Senior Unsecured,Zero-coupon bonds,Total_bonds
country,,,,,
Germany,18,16,258,25,317
Netherlands,24,17,119,8,168
France,20,16,83,26,145
Italy,24,1,100,6,131
Spain,14,11,95,1,121
United Kingdom,0,4,60,5,69
Switzerland,1,4,55,7,67
USA,1,3,37,3,44
Finland,4,0,26,1,31


Number of bonds per country. We will group countries which are similar and which have a very small number of bonds.

In [10]:
df_sm["country"].unique()

array(['Italy', 'Netherlands', 'Switzerland', 'Spain', 'BE_LU', 'Germany',
       'France', 'Non_EU', 'Ireland', 'Finland', 'Portugal', 'LT_EE',
       'Austria', 'United Kingdom', 'CZ_SK_RO', 'Sweden_Denmark'],
      dtype=object)

# Correspondence Analysis

We analyse the relationships between industry, country and type of a bond

We analyse the relationships between industry, country and type of a bond using a Correspondance Analysis. 

In [18]:
print(len(mca_ind.L)) #One factor for level
mca_ind.L  #Eigenvalues/principal inertias of each of the factors

33


array([0.3207515 , 0.23278978, 0.22631755, 0.21719662, 0.19447777,
       0.19188924, 0.16429584, 0.1502701 , 0.14970527, 0.12962739,
       0.12716017, 0.11311926, 0.10796008, 0.10390522, 0.10143871,
       0.09998701, 0.09745013, 0.09208797, 0.09178948, 0.08713953,
       0.08576074, 0.07878654, 0.07071184, 0.06665637, 0.05928371,
       0.05665613, 0.0478945 , 0.04145119, 0.03537978, 0.02829117,
       0.02356498, 0.01994727, 0.01366035])

We compute the factors of each observation and for each dummy variable and plot the first 2 factors of the correspondence analysis, for each of the dummy variables

In [15]:
Factors.T

,industry_Chemical and petrochemical industry,industry_Communication,industry_Construction and development,industry_Engineering industry,industry_Financial institutions,industry_Food industry,industry_Healthсare,industry_Information and High Technologies,industry_Light industry,industry_Media and Entertainment,industry_Oil and gas,industry_Other sectors,industry_Power,industry_Public utilities,industry_Trade and retail,industry_Transportation,country_Austria,country_BE_LU,country_CZ_SK_RO,country_Finland,country_France,country_Germany,country_Ireland,country_Italy,country_LT_EE,country_Netherlands,country_Non_EU,country_Portugal,country_Spain,country_Sweden_Denmark,country_Switzerland,country_United Kingdom,bond_types_Green bonds,bond_types_Others,bond_types_Senior Unsecured,bond_types_Zero-coupon bonds
Factor_1,-0.610791,0.123355,-0.132924,-0.400567,-0.193295,-0.838889,-0.758654,-0.334755,-0.458381,-0.744313,-0.0394232,-0.675016,1.27349,0.434626,0.121208,0.537349,0.0363928,-0.594649,0.455082,0.154708,0.0668438,-0.236016,0.582158,1.00801,2.64505,0.0123723,-0.61291,2.08136,0.384502,-0.497273,-0.990545,-0.755115,1.45759,-0.489941,-0.115363,-0.284742
Factor_2,0.530074,0.709542,-0.335301,0.139184,-0.0911403,-1.0993,-0.236551,0.295283,-0.557219,1.07937,0.737756,-1.44072,-0.355191,0.519728,0.179081,0.134675,0.781311,-1.25522,0.145441,-0.461607,-0.579055,0.276811,-0.260349,0.308002,-1.14368,-0.282168,0.441755,-0.914705,0.587139,-0.394273,-1.35848,0.554605,-0.570949,-0.560217,0.15921,-0.402475


Plot of the first 2 factors of the correspondence analysis, for each of the dummy variables

In [ ]:
labels=['Chem/Petro','Commun.', 'Construction','    Engineering', 'Financial','   Food', 'Healthсare',
       'Technology', '    Light','Entertainment', '        Oil/Gas',' Others', 'Power', 'Public utilities',
       'Trade', 'Trans.','AT', 'BE-LU', '    CZ-SK-RO','FI', 'FR', 'DE','IE', 'IT', 'LT-EE', 'NL', '  Non-EU',
       'PT', 'ES', '    SE-DK','CH', '            UK','Green bonds', '      Others','Sr. Unsec.', 'Zero-coup.']

In [ ]:
plt.subplots(figsize=(20, 12))
plt.scatter(Factors["Factor_1"], Factors["Factor_2"], s=[200]*16+[200]*16+[600]*4, color=["darkred"]*16+["green"]*16+["navy"]*4) # c=Df_PCA_Biplot["Compound"], vmin=-0.6, vmax=0.6,cmap = matplotlib.cm.get_cmap('jet_r')

plt.ylabel("Factor 2",size=40)
plt.xlabel("Factor 1",size=40)
plt.title("Relationship between Bond Type, \n Country and Industry",size=60)

for i, txt in enumerate(Factors.index):
    if i>=32:
        plt.annotate(labels[i], (Factors["Factor_1"][i]-0.2, Factors["Factor_2"][i]+0.07),size=23)
    else:
        plt.annotate(labels[i], (Factors["Factor_1"][i], Factors["Factor_2"][i]),size=20)
        
Bond_patch = mpatches.Patch(color='navy', label='Bond Types')
Country_patch = mpatches.Patch(color='green', label='Countries')
Industry_patch = mpatches.Patch(color='darkred', label='Industries')
plt.legend(handles=[Bond_patch,Country_patch,Industry_patch], fontsize=30,loc="lower left")

plt.show()

Portugal, Lithuania-Estonia, Italy and Ireland, are the countries most related to green bonds.

The Power industry is extremely related to these bonds too.

# Data Pre-processing

## Data Cleaning

Here we will:

* Change the format of numeric data
* Add a rate to floating rate bonds. We will add the mean of the rates
* Change the format of date columns and add year columns
* Obtain present value of bonds at the year 2021

Change format of numeric data

In [ ]:
df_sm["total_bond_amount"]=df_sm["total_bond_amount"].str.replace('EUR','').str.replace(',','').astype(float)

In [ ]:
df_sm["nominal_amount"]=df_sm["nominal_amount"].str.replace('EUR','').str.replace(',','').astype(float)

In [ ]:
df_sm["coupon_rate"]=df_sm["coupon_rate"].str.replace('%','').str.replace('FRN','nan').astype(float)

Add a rate to floating rate bonds (mean)

In [ ]:
df_sm["coupon_rate"][pd.isnull(df_sm["coupon_rate"])]=df_sm["coupon_rate"].mean()

Change format of date columns and add year columns

In [ ]:
df_sm['maturity_date']=pd.to_datetime(df_sm['maturity_date'])

In [ ]:
df_sm['maturity_year']=df_sm['maturity_date'].dt.year

In [ ]:
df_sm['year_created']=df_sm['year_created'].astype(int)

Obtain present value of bonds at 2021

In [ ]:
df_sm[['maturity_year','year_created']].describe()

We will now obtain a "surplus" for the company for each of the bond. For that, we obtain the value of all the interest and nominal value that the company has to pay to the bank, at the year that the bond was purchased. We then rest this amount to the total bond amount. In this way we will obtain the "surplus" of the deal for the company's side. 

After that, since different bond could have different issuing dates, we will take all the surplus values to the 2021 year. In this way we can make the surplus of the bonds comparable.

For making this process, we assign to floating rate bonds the average rate, and we considered a fix inlation rate of the one present in the euro area at the time this notebook was done, which is 1.6%.

## Annuities and present value functions

In [ ]:
def startvalue_zerobond(yr_start,yr_end,inflation,facevalue):
    diffyears=yr_end-yr_start
    totalvalue_atstart=facevalue/((1+inflation)**diffyears)
    return totalvalue_atstart

def startvalue_interest(yr_start,yr_end,inflation,interest,bondcost):
    diffyears=yr_end-yr_start
    totalvalue_atstart=((bondcost*interest)/(1+inflation))*((1-(1+inflation)**(-diffyears))/(1-(1+inflation)**(-1)))
    return totalvalue_atstart

def startvalue_completebond(yr_start,yr_end,inflation,facevalue,interest,bondcost):
    zerobond_atstart=startvalue_zerobond(yr_start,yr_end,inflation,facevalue)
    interest_atstart=startvalue_interest(yr_start,yr_end,inflation,interest,bondcost)
    all_value_atstart=interest_atstart+zerobond_atstart
    return all_value_atstart

def fromstart_topresent(yr_start,yr_present,value,inflation):
    diffyears=yr_present-yr_start
    presentvalue=value*(1+inflation)**diffyears
    return presentvalue

In [ ]:
df_sm.head(3)

## Creation of surplus at time 2021

Getting the start value of contributions to the company for the bank. We suppose an inflation of 1.6%.

In [ ]:
df_sm['Payment_atstartvalue']=startvalue_completebond(df_sm['year_created'],df_sm['maturity_year'],0.016,df_sm['nominal_amount'],
                        df_sm['coupon_rate']/100,df_sm['total_bond_amount'])

Getting the surplus at time that the bond was purchased

In [ ]:
df_sm['surplus_company']=df_sm['total_bond_amount']-df_sm['Payment_atstartvalue']

Getting the surplus value at time 2021

In [ ]:
df_sm['surplus_company_21']= fromstart_topresent(df_sm['year_created'],2021, df_sm['surplus_company']   ,0.016 )           

In [ ]:
df_sm['surplus_company_21'].describe()

In [ ]:
df_sm.sort_values(by='surplus_company_21')

# Distribution of green bonds compared to the rest of them

We will analyze the surplus in millions at 2021, instead of the surplus at 2021, just for practicality

In [ ]:
df_sm["surplus_inmill_21"]=df_sm["surplus_company_21"]/1000000
df_sm.head(4)

Distribution of the surplus at 2021 (in millions)

In [ ]:
plt.subplots(figsize=(20, 12))
sns.distplot(df_sm['surplus_inmill_21'], hist = True, kde = True,bins=30,
                 kde_kws = {'shade': True, 'linewidth': 3})#, label = bond_types)

Distribution by bond type

In [ ]:
plt.subplots(figsize=(20, 12))
sns.violinplot(x ="bond_types", y ="surplus_inmill_21", data = df_sm)# ,hue ="region", style ="event"

No signs that the bank is prioritizing green bonds so far

In [ ]:
df_sm["Ind_Green"]="No"
df_sm["Ind_Green"][df_sm["bond_types"]=="Green bonds"]="Yes"
df_sm["Ind_Green"].value_counts()

Distribution of surplus by industry and bond type

In [ ]:
plt.subplots(figsize=(10, 30))
sns.violinplot(y ="industry", x ="surplus_inmill_21", data = df_sm,width=1,dodge=True, style ="event",hue ="Ind_Green",
               split=True)# 
plt.legend(fontsize=20,title="Green \n Bond",title_fontsize=23, loc="upper right")
plt.xticks(size = 15)
plt.yticks(size = 20)
plt.xlabel("Surplus in Millions",size=25)
plt.ylabel("")
plt.title("Distribution Surplus (Millions) \n by Industry at 2021", size=25);

Surplus in communication and engineering seems bigger for green bonds than the rest, but the sample of green bonds is too small in those industries for making strong conclussions.

In [ ]:
pd.crosstab(df_sm["Ind_Green"],df_sm["industry"]).T

# Lasso

We already know that the number of green bonds is very small compared to the total number of bonds bought by the bank. B

In order to detect if the bank is helping the companies that buy green bonds in a significant way, we will do a penalized regression technique, particularly Lasso regression, on the surplus at time 2021 in millions, with the variables already analyzed as explanatory variables (industry, country, and type of bond).

We decided to use Lasso because of its nature to shrink the less important coefficients to zero. In this way, if after fitting the model, the coefficient associated to green bonds is positive, we can say that the bank is supporting the green economy.

We did not standardize our covariates since all of them are dummy variables. Standardizing will not bring any value, but it will make interpretation of the coefficients more difficult.

For hyperparameter tunning as well as model selection, we will make use of 10 fold cross validation

In [ ]:
id_bond=df_sm["ISIN"]
X=pd.get_dummies(df_sm[["industry","country","bond_types"]], prefix=None, prefix_sep='_')
y=df_sm["surplus_inmill_21"]

Just for descriptive purposes, the plain linear regression model. Thoughtful about including it in the official code/report or not.

In [ ]:
Alpha = 0
lasso_model = linear_model.Lasso(fit_intercept=True, alpha=Alpha,copy_X=True,normalize=False)
lasso_model.fit(X,y)
print('Coefficients for alpha:',str(Alpha))
pd.DataFrame(data={"Covariate":X.columns,"Coefficient":np.round(lasso_model.coef_,2)})

In [ ]:
lasso_model.intercept_

In [ ]:
lasso_model.coef_[X.columns=="bond_types_Green bonds"]

In [ ]:
lasso_model.coef_[X.columns=="industry_Power"]

In [ ]:
fig, ax = plt.subplots(1,1,sharex=True,figsize=(10,15))
y_pos = np.arange(len(X.columns))

#Lasso Regression
hedge_Lasso = lasso_model.coef_
ax.barh(y=y_pos, width=hedge_Lasso, align='center', alpha=0.9,
        color=["cadetblue"]*16+['mediumseagreen']*16+["darkred"]+["lightcoral"]*3)
ax.set_yticks(y_pos)
plt.yticks(size=15)
ax.set_yticklabels(X.columns)
ax.set_xlabel('Coefficient', size=20)
ax.set_title('Coefficients for the linear regression model', size=25);

The bank does not help at all to bonds bought to a company settled in Lithuania and Estonia.

## Impact of the alpha value to the Lasso coefficients

We set our possible alpha values from 0 to 15, because after 15 all coefficients were zero

In [ ]:
alpha = np.arange(0,15,0.01)
#alpha

Matrix that will store for each alpha, the coefficients values

In [ ]:
weights = np.zeros((len(alpha),len(X.columns)),dtype=float) #matrix of dimensions #alphas x #dummievars
weights.shape

In [ ]:
for i in np.arange(0,len(alpha)):
    lassomodelaux = linear_model.Lasso(fit_intercept=True, alpha=alpha[i],copy_X=True,normalize=False)
    lassomodelaux.fit(X,y)
    weights[i,:]= lassomodelaux.coef_  #matrix defined above starts to be filled up
    

We plot the evolution of the coefficients as the alpha increases

In [ ]:
ab = itertools.chain(["cadetblue"]*16+['mediumseagreen']*16+["darkred"]+["lightcoral"]*3)
colorlist=list(ab)

In [ ]:
ab=itertools.chain([3]*3+[7]+[3]*17+[7]+[3]*3+[7]+[3]*6+[7]+[3]*3)
sizelist=list(ab)

In [ ]:
labelsaux=['Chem/Petro','Commun.', 'Construction','Engineering', 'Financial','Food', 'Healthсare',
       'Technology', 'Light','Entertainment', 'Oil/Gas',' Others', 'Power', 'Public utilities',
       'Trade', 'Trans.','AT', 'BE-LU', 'CZ-SK-RO','FI', 'FR', 'DE','IE', 'IT', 'LT-EE', 'NL', 'Non-EU',
       'PT', 'ES', 'SE-DK','CH', 'UK','Green bonds', 'Others','Sr. Unsec.', 'Zero-coup.']

In [ ]:
plt.figure(figsize=(20,15))    
for i in np.arange(0,len(X.columns)):
    plt.semilogx(alpha[:],weights[:,i],label=labelsaux[i], linewidth=sizelist[i],
                 color=colorlist[i]) #semilog to make right values closer

plt.xlabel('Alpha Value',size=30)
plt.ylabel('Coefficient',size=30)
plt.xticks(size=20)
plt.yticks(size=20)
plt.legend(loc="lower right", ncol=5,fontsize=20);

The green bonds coefficient is in the group of coefficients that shrink faster to zero.

The coefficients that survive the most are the ones related to the engineering industry, to Germany, and to the Netherlands

## Cross validation for hyperparameter tuning and model selection

In [ ]:
alpha

In [ ]:
# use cross validation using 10 Fold cross validation
Lasso=linear_model.LassoCV(alphas=alpha,fit_intercept=True,normalize=False,cv=10)
Lasso.fit(X,y)

In [ ]:
print('Optimal Value for Alpha:{v:0.4f}'.format(v=Lasso.alpha_))
print('\n')
print('Lasso coefficients: ')
print(Lasso.coef_)
print('\n')
print('Lasso intercept:')
print(Lasso.intercept_)

The alpha that decreases the loss function the most is 1.79

In [ ]:
fig, ax = plt.subplots(1,1,sharex=True,figsize=(10,15))
y_pos = np.arange(len(X.columns))

#Lasso Regression
hedge_Lasso = Lasso.coef_
ax.barh(y=y_pos, width=hedge_Lasso, align='center', alpha=0.9,
        color=["cadetblue"]*16+['mediumseagreen']*16+["darkred"]+["lightcoral"]*3)
ax.set_yticks(y_pos)
plt.yticks(size=15)
ax.set_yticklabels(X.columns)
ax.set_xlabel('Hedge Ratio (%)')
ax.set_title('Hedging Portfolio Alpha - Ridge/Gross Exposure: ');

In [ ]:
ResultingDf=pd.DataFrame(data={"Dummy_variable":X.columns,"Coefficient":np.round(Lasso.coef_,2),"N_bonds":X.sum()})
ResultingDf=ResultingDf.reset_index()[["Dummy_variable","Coefficient","N_bonds"]]
ResultingDf.sort_values(by="Coefficient", ascending=False)

<b>Conclussion: No, the European Central bank has not been supporting the green economy when purchasing corporate
bonds</b>

Normally they will support bonds for companies whose type of industry is "Communication", "Engineering", and "Oil or Gas", and in a lesser role, companies located in "France", "Germany" and "The United Kingdom". We could corroborate with the Correspondence Analysis done, that bonds related to these factors are normally not green bonds.

Furthermore, we saw in the correspondence analysis that the industry "Power" was very related to green bonds. In this model, the coefficient "Power" is non-zero and is negative, so these corroborates again the conclussion that the European Central Bank is not supporting the green economy.

We also see that the fact of the green bond coefficient being zero could not be entirely due to the small number of green bonds, since we see that 8 of the 15 non-zero coefficients are related to variables whose number of bonds belonging to is lower than the ones for green bonds.